In [13]:
import pdfplumber
import pandas as pd

PDF = "acea_pdfs/2025_March.pdf"
PAGE = 5

with pdfplumber.open(PDF) as pdf:
    page = pdf.pages[PAGE]
    # lines 기반 (셀 경계선이 있으면 좋음)
    table = page.extract_table({
        "vertical_strategy": "lines",
        "horizontal_strategy": "lines"
    })
df_plumber = None
if table:
    h1, h2, *rows = table
    cols = [f"{(a or '').strip()} {(b or '').strip()}" or "Brand" for a, b in zip(h1, h2)]
    df_plumber = pd.DataFrame(rows, columns=cols)
print("pdfplumber 결과 (None=실패):")
display(df_plumber)


pdfplumber 결과 (None=실패):


None

In [17]:
import tabula
import pandas as pd

PDF = "acea_pdfs/2025_May.pdf"

# 1) stream 모드(셀 경계선이 없을 때)
dfs_stream = tabula.read_pdf(PDF, pages=6, multiple_tables=False, stream=True)
df_tabula_stream = dfs_stream if isinstance(dfs_stream, pd.DataFrame) else dfs_stream[0]
print("tabula-py stream 결과 (처음 5행):")
display(df_tabula_stream.head(10))

# 2) lattice 모드(셀 경계선이 있을 때, 없으면 깨짐)
dfs_lattice = tabula.read_pdf(PDF, pages=6, multiple_tables=False, lattice=True)
df_tabula_lattice = dfs_lattice if isinstance(dfs_lattice, pd.DataFrame) else dfs_lattice[0]
print("tabula-py lattice 결과 (처음 5행):")
display(df_tabula_lattice.head())


tabula-py stream 결과 (처음 5행):


,Unnamed: 0,Unnamed: 1,MAY,Unnamed: 3,Unnamed: 4,Unnamed: 5,JANUARY-MAY,Unnamed: 7
0,NaN,1% share,Units,NaN,% change,1% share,Units,% change
1,NaN,2025 2024,2025,2024,25/24,2025 2024,2025 2024,25/24
2,Volkswagen Group,27.8 27.4,"309,930","299,728",+3.4,26.7 25.6,"1,486,937 1,424,692",+4.4


tabula-py lattice 결과 (처음 5행):


,1% share\r20252024,Units\r20252024,% change\r25/24,1% share\r20252024.1,Units\r20252024.1,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,27.8,27.4,"309,930","299,728",+3.4,26.7,25.6,"1,486,937","1,424,692",+4.4
1,11.4\r6.5\r5.0\r2.4\r1.7\r0.7\r0.1,11.3\r5.7\r5.4\r1.8\r2.3\r0.8\r0.1,"127,318\r72,602\r55,651\r26,438\r18,982\r8,067...","123,067\r62,749\r59,185\r19,961\r25,447\r8,569...",+3.5\r+15.7\r-6.0\r+32.4\r-25.4\r-5.9\r+16.2,11.0\r6.0\r4.9\r2.3\r1.7\r0.7\r0.1,10.3\r5.6\r5.0\r1.6\r2.2\r0.9\r0.1,"615,646\r334,437\r270,677\r126,664\r93,836\r41...","570,968\r311,868\r279,600\r86,834\r121,491\r50...",+7.8\r+7.2\r-3.2\r+45.9\r-22.8\r-17.2\r+10.7
2,15.2,15.9,"168,839","174,131",-3.0,15.4,16.9,"859,950","939,062",-8.4
3,5.2\r3.1\r2.9\r2.2\r1.0\r0.4\r0.2\r0.1\r0.0,4.9\r3.4\r3.0\r2.6\r1.0\r0.4\r0.3\r0.4\r0.1,"57,490\r34,122\r32,161\r24,655\r11,615\r4,925\...","53,480\r37,473\r32,553\r27,862\r11,396\r4,052\...",+7.5\r-8.9\r-1.2\r-11.5\r+1.9\r+21.5\r-4.8\r-8...,5.4\r2.9\r2.8\r2.3\r1.1\r0.5\r0.2\r0.1\r0.0,5.2\r3.4\r3.2\r2.9\r1.1\r0.4\r0.3\r0.4\r0.1,"302,964\r163,743\r155,800\r126,679\r61,482\r27...","288,597\r190,574\r180,274\r158,880\r58,659\r20...",+5.0\r-14.1\r-13.6\r-20.3\r+4.8\r+33.9\r-22.8\...
4,10.0,9.7,"111,395","106,450",+4.6,10.2,9.5,"566,121","528,972",+7.0


In [10]:
import fitz  # PyMuPDF
import pandas as pd
import re

PDF = "May2025.pdf"
PAGE = 5
doc = fitz.open(PDF)
page = doc.load_page(PAGE)
text = page.get_text("text")
lines = [ln for ln in text.split("\n") if ln.strip()]
rows = [re.split(r"\s{2,}", ln.strip()) for ln in lines]
df_fitz = pd.DataFrame(rows[2:], columns=rows[0])  # 1,2번째 줄 헤더로 가정
print("PyMuPDF 결과 (처음 5행):")
display(df_fitz)


PyMuPDF 결과 (처음 5행):


,www.acea.auto
0,NEW CAR REGISTRATIONS BY MANUFACTURER
1,EU + EFTA + UK5
2,1 ACEA estimation based on total by market
3,"2 Bentley, Bugatti, Lamborghini, and MAN"
4,3 Includes Abarth
...,...
522,JANUARY-MAY
523,% share1
524,Units
525,% share1


In [29]:
import os
import calendar
import requests
import pandas as pd
import camelot
from datetime import datetime

BASE_DIR = r'C:\Users\rootn\OneDrive\Desktop\intern\car_sales'
PDF_DIR = os.path.join(BASE_DIR, 'acea_pdfs')
os.makedirs(PDF_DIR, exist_ok=True)
HEADERS = {"User-Agent": "Mozilla/5.0"}

def fetch_europe_pdf_default(url: str) -> str:
    now = datetime.now()
    year = now.year
    mon = now.month
    month_name = calendar.month_name[mon]
    fname = f"{year}_{month_name}.pdf"
    path = os.path.join(PDF_DIR, fname)
    if os.path.exists(path):
        print(f"[Info] 이미 다운로드됨: {fname}")
        return path
    resp = requests.get(url, headers=HEADERS, timeout=30)
    resp.raise_for_status()
    with open(path, 'wb') as f:
        f.write(resp.content)
    print(f"[Downloaded] {fname}")
    return path

def parse_europe_page6(pdf_path: str) -> pd.DataFrame:
    tables = camelot.read_pdf(
        pdf_path,
        pages='6',
        flavor='stream',
        strip_text='\n'
    )
    if not tables:
        raise RuntimeError(f"테이블 파싱 실패: {pdf_path}")
    df = tables[0].df

    # 2, 3번 row를 멀티헤더로, 0, 1번 row는 삭제
    header1 = df.iloc[2]
    header2 = df.iloc[3]
    data = df.iloc[4:].reset_index(drop=True)
    data.columns = pd.MultiIndex.from_arrays([header1, header2])
    return data


# 최신(이번달) PDF URL만 넣으면 됨!
PDF_URL = "https://www.acea.auto/files/Press_release_car_registrations_May_2025.pdf"

pdf_file = fetch_europe_pdf_default(PDF_URL)
df6 = parse_europe_page6(pdf_file)
result = df6.iloc[:, [0, 3]].reset_index(drop=True)

print("\n[추출된 데이터 미리보기 (상위 20행)]")
display(result)

[Info] 이미 다운로드됨: 2025_July.pdf

[추출된 데이터 미리보기 (상위 20행)]


2,,Units
3,,2025
0,Volkswagen Group,"309,930"
1,Volkswagen,"127,318"
2,Skoda,"72,602"
3,Audi,"55,651"
4,Cupra,"26,438"
5,Seat,"18,982"
6,Porsche,"8,067"
7,Others2,872
8,Stellantis,"168,839"
